# 03 - MIP Model Benchmark

This notebook implements a Mixed Integer Programming (MIP) model using PuLP to solve the Tourist Trip Design Problem as a benchmark.

## Import Libraries

In [ ]:
import sys
sys.path.append('../scripts')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from pulp import *

from data_utils import load_attractions_data
from visualization import (
    plot_route_on_map,
    plot_tour_statistics,
    create_summary_report,
    compare_algorithms
)

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## Load Data

In [ ]:
# Load attractions data
attractions = load_attractions_data('../data/sri_lanka_attractions.csv')
distance_matrix = np.load('../data/distance_matrix.npy')

scores = attractions['score'].values
visit_durations = attractions['visit_duration'].values
n_attractions = len(attractions)

print(f"Loaded {n_attractions} attractions")
attractions.head()

## Define MIP Model Parameters

In [ ]:
# Problem parameters
MAX_TIME = 24  # Maximum time available (hours)
AVG_SPEED = 50  # Average travel speed (km/h)

print(f"Maximum trip time: {MAX_TIME} hours")
print(f"Average travel speed: {AVG_SPEED} km/h")

## Build MIP Model

We formulate the Tourist Trip Design Problem as a variant of the Orienteering Problem:

**Decision Variables:**
- $x_i$ = 1 if attraction i is visited, 0 otherwise
- $y_{ij}$ = 1 if we travel from attraction i to j, 0 otherwise
- $u_i$ = position of attraction i in the tour (for subtour elimination)

**Objective:**
Maximize total satisfaction score: $\max \sum_{i} score_i \cdot x_i$

**Constraints:**
1. Time constraint: Total travel time + visit time ≤ MAX_TIME
2. Flow conservation: Each visited attraction has one incoming and one outgoing edge
3. Subtour elimination: MTZ constraints

In [ ]:
print("Building MIP model...")
start_build = time.time()

# Create the model
model = LpProblem("Tourist_Trip_Design", LpMaximize)

# Decision variables
# x[i] = 1 if attraction i is visited
x = LpVariable.dicts("visit", range(n_attractions), cat='Binary')

# y[i][j] = 1 if we travel from i to j
y = LpVariable.dicts("travel", 
                     [(i, j) for i in range(n_attractions) for j in range(n_attractions) if i != j],
                     cat='Binary')

# Position variables for subtour elimination (MTZ)
u = LpVariable.dicts("position", range(n_attractions), lowBound=0, upBound=n_attractions-1, cat='Continuous')

# Objective: Maximize total satisfaction score
model += lpSum([scores[i] * x[i] for i in range(n_attractions)]), "Total_Score"

# Constraint 1: Time constraint
total_time = lpSum([visit_durations[i] * x[i] for i in range(n_attractions)]) + \
             lpSum([distance_matrix[i][j] / AVG_SPEED * y[(i, j)] 
                    for i in range(n_attractions) for j in range(n_attractions) if i != j])
model += total_time <= MAX_TIME, "Time_Limit"

# Constraint 2: Flow conservation
# If an attraction is visited, it must have exactly one incoming and one outgoing edge
for i in range(n_attractions):
    # Outgoing edges
    model += lpSum([y[(i, j)] for j in range(n_attractions) if i != j]) == x[i], f"Outflow_{i}"
    # Incoming edges
    model += lpSum([y[(j, i)] for j in range(n_attractions) if i != j]) == x[i], f"Inflow_{i}"

# Constraint 3: Subtour elimination (MTZ constraints)
for i in range(n_attractions):
    for j in range(n_attractions):
        if i != j and i > 0 and j > 0:  # Exclude depot (attraction 0)
            model += u[i] - u[j] + n_attractions * y[(i, j)] <= n_attractions - 1, f"MTZ_{i}_{j}"

end_build = time.time()
print(f"Model built in {end_build - start_build:.2f} seconds")
print(f"Number of variables: {len(model.variables())}")
print(f"Number of constraints: {len(model.constraints)}")

## Solve the MIP Model

In [ ]:
print("Solving MIP model...")
start_solve = time.time()

# Solve the model
model.solve(PULP_CBC_CMD(msg=1, timeLimit=300))  # 5 minute time limit

end_solve = time.time()
computation_time = end_solve - start_solve

print(f"\nSolution status: {LpStatus[model.status]}")
print(f"Computation time: {computation_time:.2f} seconds")
print(f"Objective value (Total Score): {value(model.objective):.2f}")

## Extract Solution

In [ ]:
# Extract visited attractions
visited_attractions = [i for i in range(n_attractions) if value(x[i]) > 0.5]

print(f"Number of attractions visited: {len(visited_attractions)}")
print(f"\nVisited attractions:")
for idx in visited_attractions:
    print(f"  - {attractions.iloc[idx]['name']} (Score: {scores[idx]})")

# Extract tour sequence
# Build adjacency list
tour_edges = [(i, j) for i in range(n_attractions) for j in range(n_attractions) 
              if i != j and value(y[(i, j)]) > 0.5]

print(f"\nTour edges: {tour_edges}")

# Reconstruct tour sequence
if visited_attractions:
    # Create adjacency dictionary
    next_attraction = {i: j for i, j in tour_edges}
    
    # Start from first attraction
    tour_sequence = []
    if next_attraction:
        current = list(next_attraction.keys())[0]
        tour_sequence.append(current)
        
        visited_set = {current}
        while current in next_attraction and next_attraction[current] not in visited_set:
            current = next_attraction[current]
            tour_sequence.append(current)
            visited_set.add(current)
    else:
        tour_sequence = visited_attractions
    
    print(f"\nTour sequence:")
    for i, idx in enumerate(tour_sequence):
        print(f"{i+1}. {attractions.iloc[idx]['name']}")
else:
    tour_sequence = []

## Verify Solution

In [ ]:
# Calculate actual time used
if tour_sequence:
    total_visit_time = sum([visit_durations[i] for i in tour_sequence])
    total_travel_time = 0
    total_travel_dist = 0
    
    for i in range(len(tour_sequence) - 1):
        dist = distance_matrix[tour_sequence[i], tour_sequence[i+1]]
        total_travel_dist += dist
        total_travel_time += dist / AVG_SPEED
    
    total_time = total_visit_time + total_travel_time
    total_score = sum([scores[i] for i in tour_sequence])
    
    print("Solution Verification:")
    print(f"Total visit time: {total_visit_time:.2f} hours")
    print(f"Total travel distance: {total_travel_dist:.2f} km")
    print(f"Total travel time: {total_travel_time:.2f} hours")
    print(f"Total time: {total_time:.2f} hours (limit: {MAX_TIME} hours)")
    print(f"Total score: {total_score:.2f}")
    print(f"\nTime constraint satisfied: {total_time <= MAX_TIME}")

## Visualize MIP Solution

In [ ]:
if tour_sequence:
    # Plot tour statistics
    fig = plot_tour_statistics(attractions, tour_sequence, distance_matrix)
    plt.show()
    
    # Create interactive map
    tour_map = plot_route_on_map(attractions, tour_sequence, output_file='../data/mip_tour_map.html')
    print("\nInteractive map created! Open ../data/mip_tour_map.html in a browser to view.")
    
    # Generate summary report
    report = create_summary_report(attractions, tour_sequence, distance_matrix, algorithm_name='MIP (PuLP)')
    print(report)
    
    # Save report
    with open('../data/mip_tour_report.txt', 'w') as f:
        f.write(report)
    print("Report saved to ../data/mip_tour_report.txt")

## Compare with Genetic Algorithm

In [ ]:
# Load GA results
try:
    ga_results = np.load('../data/ga_results.npy', allow_pickle=True).item()
    
    # Prepare comparison data
    comparison_results = {
        'Genetic Algorithm': {
            'score': ga_results['best_fitness'],
            'computation_time': ga_results['computation_time']
        },
        'MIP (PuLP)': {
            'score': value(model.objective) if tour_sequence else 0,
            'computation_time': computation_time
        }
    }
    
    # Plot comparison
    fig = compare_algorithms(comparison_results)
    plt.show()
    
    # Print comparison
    print("\nAlgorithm Comparison:")
    print("=" * 60)
    print(f"{'Algorithm':<20} {'Score':<15} {'Time (s)':<15}")
    print("=" * 60)
    for algo, results in comparison_results.items():
        print(f"{algo:<20} {results['score']:<15.2f} {results['computation_time']:<15.2f}")
    print("=" * 60)
    
except:
    print("GA results not found. Run notebook 02 first to generate GA results.")

## Save MIP Results

In [ ]:
# Save MIP results
if tour_sequence:
    mip_results = {
        'algorithm': 'MIP (PuLP)',
        'tour_sequence': tour_sequence,
        'total_score': value(model.objective),
        'computation_time': computation_time,
        'status': LpStatus[model.status],
        'parameters': {
            'max_time': MAX_TIME,
            'avg_speed': AVG_SPEED
        }
    }
    
    np.save('../data/mip_results.npy', mip_results)
    print("Results saved to ../data/mip_results.npy")

## Analysis and Insights

### MIP Model Advantages:
- Provides optimal or near-optimal solutions (with guarantees)
- Deterministic results
- Can provide optimality gaps

### MIP Model Limitations:
- Computational complexity grows with problem size
- May require significant time for large instances
- Memory intensive for large problems

### When to Use MIP vs GA:
- **MIP**: Smaller problems (<50 attractions), when optimality is critical
- **GA**: Larger problems, when good solutions are needed quickly, real-time applications

## Next Steps

In the next notebook, we will create comprehensive visualizations and analyze the results from both algorithms.